In [ ]:
## Basic exploration

### Import modules

In [ ]:
import os, sys, time, random, math
import tarfile, zipfile  # Work with compressed files

import numpy as np     # Linear algebra
import pandas as pd    # Data processing

from IPython.display import display, Image

from subprocess import check_output
print(check_output(["ls", "../../input"]).decode("utf8"))

### Load data

In [ ]:
data = pd.read_csv("../../input/train.csv")
test_data = pd.read_csv("../../input/test.csv")

### Examine shape of datasets

In [ ]:
print ("Dataset has {} samples with {} features each.".format(*data.shape))
data.info()
display(data.head(5))

print ("Dataset has {} samples with {} features each.".format(*test_data.shape))
test_data.info()
display(test_data.head(5))

### Examine distrbutions of datasets

In [ ]:
cols = data.columns
cats = [c for c in cols if "cat" in c]
conts = [c for c in cols if "cont" in c]

print('Total {} columns. {} category columns, {} continuous columns'.format(len(cols), len(cats), len(conts)))
print('Avg loss = {:.3f}'.format(data['loss'].mean()))
for c in conts:
    print('Average value of {} = {:.3f}'.format(c, data[c].mean()) )
data['loss'].describe()

### Plot loss distributions

In [ ]:
from ggplot import *
ggplot(aes(x = 'loss'), data) + geom_density()

### Check if category values of test_data appear in train_data

In [ ]:
for c in cats[0:]:
    vals_d = data[c].unique()
    vals_t = test_data[c].unique()
    for v in vals_t:
        if v not in vals_d:
            print(c, v)